# Transformer

In [1]:
import math
import pandas
import torch
from torch import nn
from d2l import torch as d2l

# 基于位置的前馈网络

In [3]:
class PosiTionWiseFFN(nn.Module):
    def __init__(self, ffn_num_input, ffn_num_hiddens, ffn_num_outputs, **kwargs):
        super(PosiTionWiseFFN, self).__init__(**kwargs)
        self.dense1 = nn.Linear(ffn_num_input, ffn_num_hiddens)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(ffn_num_hiddens, ffn_num_outputs)

    def forward(self, X):
        return self.dense2(self.relu(self.dense1(X)))

## 改变张量的最里层维度的尺寸

In [7]:
ffn = PosiTionWiseFFN(4, 4, 8)
ffn.eval()
ffn(torch.ones(2, 3, 4))[0]

tensor([[-0.4196,  0.1387,  0.3948,  0.0161, -0.0943,  0.4745,  0.4312,  0.3951],
        [-0.4196,  0.1387,  0.3948,  0.0161, -0.0943,  0.4745,  0.4312,  0.3951],
        [-0.4196,  0.1387,  0.3948,  0.0161, -0.0943,  0.4745,  0.4312,  0.3951]],
       grad_fn=<SelectBackward0>)

In [ ]:
# 对比不同维度的层归一化和批量归一化的效果